### 5차시 : 크롤링 후 excel 저장하기
- 과제: 교수님 사이트 외에서 크롤링 결과값 가져오기
- title + 링크(주소) 갖고와서 1:1 출력하기
- 스크랩핑 제한이 있으면 보통 API를 열어준다. 이건 다음시간에 찾아해볼 것

#### 학습 목표
- 기사 타이틀과 링크 가져오기
- 여러 페이지로 나눠져있는 검색 결과물 스크랩핑하기
- 가져온 자료를 엑셀 파일로 만들기

In [1]:
# get url & Parsing
# 사이트는 folha de Sao paulo
import requests
from bs4 import BeautifulSoup
url = 'https://search.folha.uol.com.br/search?q=coreia+do+sul&periodo=todos&sd=&ed=&site=todos'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

C:\Users\bestl\anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# orderedlist 가져와
sections = soup.find('ol', class_='u-list-unstyled c-search')
# h2 태그를 갖고와
titles = sections.find_all('h2')
# a태그를 전부가져와: href = True: a태그지만 링크가 없는 애들이 종종 있음. 
# 지금은 잘 돌아가지만 간혹 저 href = True가 없으면 주소가 없는 a태그까지 다 갖고오는 경우도 있다.
links = sections.find_all('a', href=True) 

- 알아야할 함수: if, for, while

In [3]:
### 오늘의핵심 코드#####: 타이틀과 링크 가져와서 출력하기
# 타이틀과 링크 둘다 가져와야함: min(len(titles),len(links))로 개수의 최소 길이를 골라서 작은쪽으로 맞춰줌(헛돌아 에러나는 것 방지)
for article in range(min(len(titles), len(links))):
    # get_text:html 태그 빼주고 텍스트만 갖고오게함. strip으로 좌우 공백 제거
    title = titles[article].get_text().strip()
    # 그 article의 링크를 가져와
    link = links[article]['href']
    # f포맷 사용하여 출력
    print(f"제목: {title}\n링크: {link}\n")

제목: 3 curiosos enigmas sobre o tempo (e por que há quem viva nos séculos 15 e 30 simultaneamente)
링크: https://www1.folha.uol.com.br/ciencia/2024/03/3-curiosos-enigmas-sobre-o-tempo-e-por-que-ha-quem-viva-nos-seculos-15-e-30-simultaneamente.shtml

제목: Argentina negocia compra de 24 caças F-16 usados da Dinamarca
링크: https://www1.folha.uol.com.br/ciencia/2024/03/3-curiosos-enigmas-sobre-o-tempo-e-por-que-ha-quem-viva-nos-seculos-15-e-30-simultaneamente.shtml

제목: A disputa geopolítica pelas riquezas do fundo do oceano
링크: https://www1.folha.uol.com.br/mundo/2024/03/argentina-negocia-compra-de-24-cacas-f-16-usados-da-dinamarca.shtml

제목: Voluntários coletam lixo em praia de Taiwan; veja fotos de hoje
링크: https://www1.folha.uol.com.br/mundo/2024/03/argentina-negocia-compra-de-24-cacas-f-16-usados-da-dinamarca.shtml

제목: Veto da Rússia extingue grupo que fiscaliza sanções da ONU a programa nuclear da Coreia do Norte
링크: https://www1.folha.uol.com.br/ambiente/2024/03/a-disputa-geopolitica-pe

In [4]:
# 인덱스까지 붙이기
url = 'https://search.folha.uol.com.br/search?q=coreia+do+sul&periodo=todos&sd=&ed=&site=todos'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
sections = soup.find('ol', class_='u-list-unstyled c-search')
titles = sections.find_all('h2')
links = sections.find_all('a', href=True) # href를 포함한 a 태그만 가져오기

# 타이틀과 링크를 순서번호와 함께 출력 하기
for index in range(min(len(titles), len(links))):
    title = titles[index].get_text().strip()
    link = links[index]['href']
    print(f"{index + 1}.제목: {title}\n링크: {link}\n")

1.제목: 3 curiosos enigmas sobre o tempo (e por que há quem viva nos séculos 15 e 30 simultaneamente)
링크: https://www1.folha.uol.com.br/ciencia/2024/03/3-curiosos-enigmas-sobre-o-tempo-e-por-que-ha-quem-viva-nos-seculos-15-e-30-simultaneamente.shtml

2.제목: Argentina negocia compra de 24 caças F-16 usados da Dinamarca
링크: https://www1.folha.uol.com.br/ciencia/2024/03/3-curiosos-enigmas-sobre-o-tempo-e-por-que-ha-quem-viva-nos-seculos-15-e-30-simultaneamente.shtml

3.제목: A disputa geopolítica pelas riquezas do fundo do oceano
링크: https://www1.folha.uol.com.br/mundo/2024/03/argentina-negocia-compra-de-24-cacas-f-16-usados-da-dinamarca.shtml

4.제목: Voluntários coletam lixo em praia de Taiwan; veja fotos de hoje
링크: https://www1.folha.uol.com.br/mundo/2024/03/argentina-negocia-compra-de-24-cacas-f-16-usados-da-dinamarca.shtml

5.제목: Veto da Rússia extingue grupo que fiscaliza sanções da ONU a programa nuclear da Coreia do Norte
링크: https://www1.folha.uol.com.br/ambiente/2024/03/a-disputa-geop

In [5]:
# pagenation: 여러 페이지 한번에 추출하기
# folha에서 Coreia do Sul로 2024.1월부터 3.28일까지 검색한 결과 페이지
base_url = 'https://search.folha.uol.com.br/search?q=coreia%20do%20sul&site=todos&sd=01%2F01%2F2024&ed=28%2F03%2F2024&periodo=personalizado'
total_results = 182 # 총 결과 수
results_per_page = 25 # 페이지 당 결과 수

# for은 0,10이면 0~9까지 돌기때문에 +1해서 마지막페이지 보존
# result_per_page: 25개(한페이지 당 있는 개수. 위에서 25개로 정의)
for sr in range(1, total_results + 1, results_per_page):
    # sr 값을 이용해 각 페이지의 URL 생성
    url = f"{base_url}&sr={sr}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    sections = soup.find('ol', class_='u-list-unstyled c-search')
    # 기사 제목 추출
    titles = sections.find_all('h2')
    
    # 출력문. enumerate가 튜플형태로 반환해준다.(두요소씩 돌게 해줌)
    for index, title in enumerate(titles, start=sr):
        print(f"{index}. {title.get_text().strip()}")

1. Empresa oferece US$ 75 mil a funcionário que tiver filho na Coreia do Sul
2. Teles buscam pedágio da internet via Anatel
3. Japão: o país onde se vendem mais fraldas para adultos do que para bebês
4. A seleção reata laços com a torcida
5. EUA e Europa debatem fim de subsídios ao petróleo, mas consenso é desafio
6. Disputa das montadoras atrasa principais pontos do programa Mover
7. Ex-presidente da Associação Chinesa de Futebol é condenado à prisão perpétua
8. O que está por trás da crise do k-pop
9. Como motéis se tornaram uma particularidade do Brasil
10. Conselho de Segurança aprova 1ª resolução que demanda cessar-fogo imediato em Gaza
11. Brasileiro vira dono de hotel em NY na Justiça e mora 5 anos de graça até ser preso
12. Engenheiros juniores podem ganhar US$ 1 milhão para desenvolver IAs
13. Brasil é uma aberração nas opiniões sobre aborto e drogas
14. O mundo em 15 fotos; Mural de Banksy, bombardeios em Kiev e Kate Middleton
15. Rússia e China vetam resolução dos EUA que de

### excel 관련 함수
- openpyxl
- !pip install openpyxl
- 파일 이름과 시트명은 다르다

In [17]:
# 주로: import openpyxl as px

In [6]:
# import 외의 방법. 컴퓨터에서 엑셀이 없는 상태로 엑셀을 저장하겠다면?!
from openpyxl import Workbook
# wb라는 엑셀 워크북 생성
wb = Workbook()
# wb라는 워크북에서 디폴트 시트 활성화하고 변수로 지정하기. 
# 변수명은 한글 불가
ws = wb.active
# 시트 이름 바꾸기. default = sheet1임. 이름 바꿔주기
ws.title = '한국관련 기사'
# 시트 A1 셀에 'coreia' 입력하기
ws['A1'] = 'coreia'
# 시트에 A1-C1셀에 데이타 입력: 지우지 않게 append. write하면 다 지워진다.
ws.append (['Coreia', 'Brasil', 'Portugal'])

# 워크북을 news_coreia.xlsx 로 저장
wb.save('news_coreia.xlsx')
# 워크북 닫기
wb.close()

In [7]:
import os
print(os.getcwd())


C:\Users\bestl\OneDrive\바탕 화면\24-1\포르투갈어 데이터마이닝


In [8]:
import pandas as pd

# 엑셀 파일 읽어오기
df = pd.read_excel('news_coreia.xlsx')

# 데이터프레임 출력
print(df.head(10))

   coreia Unnamed: 1 Unnamed: 2
0  Coreia     Brasil   Portugal


### 주의사항
- openpyxl: 저장 그냥 하면 덮어써버림
    - 때문에 write보다 append를 이용할 필요가 있다. 

In [9]:
# 함수화 코드
from openpyxl import Workbook
def excel_template(filename, sheetname, datalist):
    wb = openpyxl.Workbook()
    ws = wb.active
    # 셀dimension 넓혀주기
    ws.column_dimensions['A'].width = 100
    ws.column_dimensions['B'].width = 150
    # 시트네임이 없지 않으면 ws.title을 시트이름으로 주기ㅣ
    if sheetname != '':
        ws.title = sheetname
    for item in datalist:
        ws.append(item)
    wb.save(filename)
    wb.close()

In [10]:
import openpyxl
import requests
from bs4 import BeautifulSoup
def excel_template(filename, sheetname, datalist):
    url = 'https://search.folha.uol.com.br/search?q=coreia+do+sul&periodo=personalizado&sd=01%2F01%2F2024&ed=28%2F03%2F2024&site=todos'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles_container = soup.select('ol')
    head_lines = []
    
for item in articles_container:
    for article in item.select('li'):
        head_line = article.select_one('h2').get_text().strip()
        link = article.select_one('a').get('href')
        head_lines.append([head_line, link])
                                  
excel_template('news.xlsx', 'coreia', head_lines)

NameError: name 'articles_container' is not defined

In [11]:
# 수정 코드
import openpyxl
import requests
from bs4 import BeautifulSoup

def excel_template(filename, sheetname, datalist):
    wb = openpyxl.Workbook()  # 새로운 워크북 생성
    ws = wb.active  # 기본 시트 활성화
    
    # 시트 이름 설정
    ws.title = sheetname
    
    # 데이터 추가
    for data in datalist:
        ws.append(data)
    
    # 파일 저장
    wb.save(filename)
    wb.close()

# 웹 페이지에서 기사 헤드라인과 링크 수집
url = 'https://search.folha.uol.com.br/search?q=coreia+do+sul&periodo=personalizado&sd=01%2F01%2F2024&ed=28%2F03%2F2024&site=todos'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
articles_container = soup.select('ol')
# title, link쌍으로 되어있음. 
head_lines = []

for item in articles_container:
    for article in item.select('li'):
        head_line = article.select_one('h2').get_text().strip()
        link = article.select_one('a').get('href')
        head_lines.append([head_line, link])
                                  
# 엑셀 파일 생성
excel_template('news.xlsx', 'coreia', head_lines)


In [15]:
import pandas as pd

# 엑셀 파일 읽어오기
df = pd.read_excel('news.xlsx')

# 데이터프레임 출력
print(df.head(10))

  Empresa oferece US$ 75 mil a funcionário que tiver filho na Coreia do Sul  \
0        Teles buscam pedágio da internet via Anatel                          
1  Japão: o país onde se vendem mais fraldas para...                          
2                A seleção reata laços com a torcida                          
3  EUA e Europa debatem fim de subsídios ao petró...                          
4  Disputa das montadoras atrasa principais ponto...                          
5  Ex-presidente da Associação Chinesa de Futebol...                          
6              O que está por trás da crise do k-pop                          
7  Como motéis se tornaram uma particularidade do...                          
8  Conselho de Segurança aprova 1ª resolução que ...                          
9  Brasileiro vira dono de hotel em NY na Justiça...                          

  https://www1.folha.uol.com.br/mercado/2024/03/empresa-oferece-us-75-mil-a-funcionario-que-tiver-filho-na-coreia-do-sul.shtml  
0

In [17]:
# 컬럼명 추가하기
df

,Empresa oferece US$ 75 mil a funcionário que tiver filho na Coreia do Sul,https://www1.folha.uol.com.br/mercado/2024/03/empresa-oferece-us-75-mil-a-funcionario-que-tiver-filho-na-coreia-do-sul.shtml
0,Teles buscam pedágio da internet via Anatel,https://www1.folha.uol.com.br/colunas/painelsa...
1,Japão: o país onde se vendem mais fraldas para...,https://www1.folha.uol.com.br/mercado/2024/03/...
2,A seleção reata laços com a torcida,https://www1.folha.uol.com.br/colunas/jucakfou...
3,EUA e Europa debatem fim de subsídios ao petró...,https://www1.folha.uol.com.br/mercado/2024/03/...
4,Disputa das montadoras atrasa principais ponto...,https://www1.folha.uol.com.br/mercado/2024/03/...
5,Ex-presidente da Associação Chinesa de Futebol...,https://www1.folha.uol.com.br/esporte/2024/03/...
6,O que está por trás da crise do k-pop,https://f5.folha.uol.com.br/musica/2024/03/o-q...
7,Como motéis se tornaram uma particularidade do...,https://f5.folha.uol.com.br/voceviu/2024/03/co...
8,Conselho de Segurança aprova 1ª resolução que ...,https://www1.folha.uol.com.br/mundo/2024/03/co...
9,Brasileiro vira dono de hotel em NY na Justiça...,https://www1.folha.uol.com.br/mundo/2024/03/br...


### 결과물 21p 참고
- news.xlsx로 확인

In [13]:
#파일 읽어들이는 법
import openpyxl
wb = openpyxl.load_workbook('news.xlsx')
# print(wb.sheetnames)
ws = wb.active
for index, item in enumerate(ws.rows):
    print(index+1, item[0].value, item[1].value)

1 Empresa oferece US$ 75 mil a funcionário que tiver filho na Coreia do Sul https://www1.folha.uol.com.br/mercado/2024/03/empresa-oferece-us-75-mil-a-funcionario-que-tiver-filho-na-coreia-do-sul.shtml
2 Teles buscam pedágio da internet via Anatel https://www1.folha.uol.com.br/colunas/painelsa/2024/03/ministro-esconde-jogo-e-teles-buscam-pedagio-da-internet-via-anatel.shtml
3 Japão: o país onde se vendem mais fraldas para adultos do que para bebês https://www1.folha.uol.com.br/mercado/2024/03/japao-o-pais-onde-se-vendem-mais-fraldas-para-adultos-do-que-para-bebes.shtml
4 A seleção reata laços com a torcida https://www1.folha.uol.com.br/colunas/jucakfouri/2024/03/a-selecao-reata-lacos-com-a-torcida.shtml
5 EUA e Europa debatem fim de subsídios ao petróleo, mas consenso é desafio https://www1.folha.uol.com.br/mercado/2024/03/eua-e-europa-debatem-fim-de-subsidios-a-petroleo-mas-consenso-e-desafio.shtml
6 Disputa das montadoras atrasa principais pontos do programa Mover https://www1.folha.u